In [ ]:
# This document/.ipynb file was originaly created by Sacha Dubrulle in Python 2 
# and further worked upon and converted to Python 3 by Thomas Valcke.
# This notebook has been converted to Python 3 because the syntax changed with Python 3.

# When run on the server there was 1 Cell that did not work RF_classification again a random forest model
# ==> get_ipython().run_cell_magic('time', '', 'RF_model_all = RF_classification(model_description = "All features RF",\n                    df = ML_BT,\n                    cat_columns = text_cat_cols + non_text_cat_cols, \n                    num_columns = text_num_cols + non_text_num_cols)\nRF_model_text = RF_classification(model_description = "Text features RF",\n                  df = ML_BT,\n                  cat_columns = text_cat_cols, \n                  num_columns = text_num_cols)\n\nRF_model_all = RF_classification(model_description = "Non-text features RF",\n                    df = ML_BT,\n                    cat_columns = non_text_cat_cols, \n                    num_columns = non_text_num_cols)')
# The reason for this cell not working is still unkown.

# <span style = 'color:darkred'> Imports </span style>

In [1]:
import os
import sys
os.environ['SPARK_HOME'] = 'D:/School/STAGE_BP/spark' # your spark location

In [3]:
################################ JUPYTER ###################################
from IPython.core.display import display, HTML
from IPython.display import Image
display(HTML("<style>.container { width:100% !important; }</style>"))

################################ SPARK ###################################

import pyspark

from pyspark.sql import SQLContext, SparkSession

try:
    sc = pyspark.SparkContext(appName = 'App')
    spark = SparkSession(sparkContext=sc)
    print("SparkSession initialized")
except ValueError:
    print("SparkSession already initialized")
    
from pyspark.sql.functions import udf, explode, lower, regexp_extract, split, col, regexp_replace
from pyspark.sql.types import StringType, TimestampType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.ml.feature import Binarizer, StringIndexer, StringIndexerModel, OneHotEncoder, MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

################################ PYTHON ###################################
    
from datetime import datetime
import re
import pandas as pd
pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_columns', 200)

import numpy as np
import urllib.request, urllib.error, urllib.parse
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, WeekdayLocator, DateFormatter
from dateutil.relativedelta import relativedelta

SparkSession initialized


In [4]:
# Read prepared data from data preperation in parquet format
ML_BT = spark.read.parquet("D:/School/STAGE_BP/ipo/ML_features.parquet")

#ML_BT.drop("cashtags", "hashtags").repartition(1).write.csv("MLBT.csv")


In [5]:
#ML_BT.show()

In [6]:
#ML_BT.select("contains_cashtag").show(10, truncate =  False)

In [7]:
ML_BT.sample(withReplacement=False, fraction = 0.1).select("text").show(10, truncate = False)



+--------------------------------------------------------------------------------------------------------------+
|text                                                                                                          |
+--------------------------------------------------------------------------------------------------------------+
|KAYA PAS HOKAGE KETIGA LAWAN OROCHIMARU https://t.co/0g8lxrdkRo                                               |
|RT https://t.co/pG2TepSKoC                                                                                    |
|my right leg keeps popping out of place and my abs ache.....I love dance so much                              |
|RT @_velvetbullets: Twitter needs an edit button. I stay looking uneducated af bc of typos smh                |
|RT @wiwer77: The Humanist's Emerging Role in #BigData 
https://t.co/dIvlvGYkXv via @datanami                  |
|@mohdhaider1984 @uniquepersonaap @NitinTyagiAAP 

Hum India Ko mehnat se aage lekar jaana chaht

In [8]:

ML_BT.select("text").show(10, truncate =  False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|@leoradiofobia como ? que c? faz pros olhos n?o fritarem depois de horas vendo uma timeline de edi??o? Puta merda, viu?!                    |
|RT @its_branndon: Legit this is karr &amp; Kim idc https://t.co/HdqtqNhxSN                                                                  |
|@maryterry2015 You've been quoted in my #Storify story "NRMS SS STAAR Chat #2 - A New Nation" https://t.co/LFbpJu2iUh                       |
|I cant believe wtf i just heard i dare you have that nigga say somethin to me im knockin him tf out                                         |

# <span style = 'color:darkred'> Classical ensemble using Spark </span style>

In [9]:
def data_preparation_to_Pandas(df, cat_columns, num_columns, output_column = "retweet_binary"):
    stringIndexers = []
    indexed_cols = []

    encoders = []
    encoded_cols = []

    for column in cat_columns:
        inCol= column
        outCol = column + "indexed"
        stringIndexer = StringIndexer(inputCol = inCol, outputCol=outCol)

        stringIndexers.append(stringIndexer)
        indexed_cols.append(outCol)

    for column in indexed_cols:
        inCol= column
        outCol = column + "encoded"
        encoder = OneHotEncoder(inputCol=inCol, outputCol=outCol)

        encoders.append(encoder)
        encoded_cols.append(outCol)

    binarizer = Binarizer(inputCol = "retweet_count_double", outputCol = output_column, threshold=0.9)

    assembler = VectorAssembler(inputCols= num_columns + encoded_cols, outputCol="features")
    
    pipeline = Pipeline(stages= stringIndexers + encoders + [assembler] + [binarizer])
    pipelineModel = pipeline.fit(df)

    transformed_df = (pipelineModel.transform(df)
                                   .select("features", 
                                            col("retweet_binary").cast("integer"), 
                                            "retweet_count")
                     )
    
    return transformed_df.toPandas()

# <span style = 'color:darkred'> Random Forest

In [10]:
results  = pd.DataFrame(columns = ["algorithm", "train AUC", "test AUC"])
results

,algorithm,train AUC,test AUC


In [11]:
def RF_classification(model_description, df, cat_columns, num_columns, output_column = "retweet_binary"):
    stringIndexers = []
    indexed_cols = []
    encoders = []
    encoded_cols = []

    for column in cat_columns:
        inCol= column
        outCol = column + "indexed"
        stringIndexer = StringIndexer(inputCol = inCol, outputCol=outCol)
        stringIndexers.append(stringIndexer)
        indexed_cols.append(outCol)

    for column in indexed_cols:
        inCol= column
        outCol = column + "encoded"
        encoder = OneHotEncoder(inputCol=inCol, outputCol=outCol)
        encoders.append(encoder)
        encoded_cols.append(outCol)

    binarizer = Binarizer(inputCol = "retweet_count_double", outputCol = output_column, threshold=0.9)
    assembler = VectorAssembler(inputCols= num_columns + encoded_cols, outputCol="features")
    
    pipeline = Pipeline(stages= stringIndexers + encoders + [assembler] + [binarizer])
    pipelineModel = pipeline.fit(df)
    
    transformed_df = (pipelineModel.transform(df)
                                   .select("features", col("retweet_binary").cast("integer"), "retweet_count")
                     )
    
    train, validation, test = transformed_df.randomSplit([0.8, 0.1, 0.1], seed = 42)
    
    rf = RandomForestClassifier(numTrees=500, labelCol="retweet_binary")
    rf_model = rf.fit(train)

    train_predictions = rf_model.transform(train)
    validation_predictions = rf_model.transform(validation)
    
    AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="retweet_binary", metricName="areaUnderROC")

    AUC_train = AUC_evaluator.evaluate(train_predictions)
    AUC_validation = AUC_evaluator.evaluate(validation_predictions)
    print("###################" + model_description.upper() + "##################")
    print("TRAIN AUC:       {}".format(AUC_train))
    print("VALIDATION AUC:        {}".format(AUC_validation))
    
    global results
    results = results.append({"algorithm": model_description, "train AUC": AUC_train, "test AUC": AUC_validation}, ignore_index=True)
    
    return rf_model

In [12]:
non_text_num_cols = ["user_seniority", "user_favourites_count", "user_friends_count", "user_followers_count"]
non_text_cat_cols = ["day_of_week"]
text_num_cols = ["cashtag_count", "hashtag_count"]
text_cat_cols = ["lang", "contains_cashtag", "contains_cashtag_with_spaces", "contains_ipo", "contains_hashtags", "contains_url"]
dependent_variable = ["retweet_count"]
text = ["clean_text"]


In [16]:
#df.show()

In [17]:
get_ipython().run_cell_magic('time', '', 'RF_model_all = RF_classification(model_description = "All features RF",\n                    df = ML_BT,\n                    cat_columns = text_cat_cols + non_text_cat_cols, \n                    num_columns = text_num_cols + non_text_num_cols)\nRF_model_text = RF_classification(model_description = "Text features RF",\n                  df = ML_BT,\n                  cat_columns = text_cat_cols, \n                  num_columns = text_num_cols)\n\nRF_model_all = RF_classification(model_description = "Non-text features RF",\n                    df = ML_BT,\n                    cat_columns = non_text_cat_cols, \n                    num_columns = non_text_num_cols)')


Py4JJavaError: An error occurred while calling o623.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 26.0 failed 1 times, most recent failure: Lost task 11.0 in stage 26.0 (TID 251, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 1
	at org.apache.spark.ml.linalg.DenseVector.apply(Vectors.scala:462)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:82)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:81)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:193)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(Shuffle

# <span style = 'color:darkred'> Linear regression model

In [20]:
from pyspark.ml.classification import LogisticRegression

In [21]:
def LR_classification(model_description, df, cat_columns, num_columns, output_column = "retweet_binary"):
    stringIndexers = []
    indexed_cols = []

    encoders = []
    encoded_cols = []

    for column in cat_columns:
        inCol= column
        outCol = column + "indexed"
        stringIndexer = StringIndexer(inputCol = inCol, outputCol=outCol)

        stringIndexers.append(stringIndexer)
        indexed_cols.append(outCol)

    for column in indexed_cols:
        inCol= column
        outCol = column + "encoded"
        encoder = OneHotEncoder(inputCol=inCol, outputCol=outCol)

        encoders.append(encoder)
        encoded_cols.append(outCol)

    binarizer = Binarizer(inputCol = "retweet_count_double", outputCol = output_column, threshold=0.9)

    assembler = VectorAssembler(inputCols= num_columns + encoded_cols, outputCol="features")
    
    pipeline = Pipeline(stages= stringIndexers + encoders + [assembler] + [binarizer])
    pipelineModel = pipeline.fit(df)

    transformed_df = (pipelineModel.transform(df)
                                   .select("features", 
                                            col("retweet_binary").cast("integer"), 
                                            "retweet_count")
                     )
    
    transformed_df = transformed_df
    
    train, validation, test = transformed_df.randomSplit([0.8, 0.1, 0.1], seed = 42)
    
    lr = LogisticRegression(labelCol="retweet_binary")
    lr_model = lr.fit(train)

    train_predictions = lr_model.transform(train)
    validation_predictions = lr_model.transform(validation)
    
    AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="retweet_binary", metricName="areaUnderROC")

    AUC_train = AUC_evaluator.evaluate(train_predictions)
    AUC_validation = AUC_evaluator.evaluate(validation_predictions)
    print("###################" + model_description.upper() + "##################")
    print("TRAIN AUC:       {}".format(AUC_train))
    print("TEST AUC:        {}".format(AUC_validation))
    
    global results
    results = results.append({"algorithm": model_description, "train AUC": AUC_train, "test AUC": AUC_validation}, ignore_index=True)
    
    return lr_model
    #return encoders

In [22]:
# Because the parquet in data preperation is not created correctly with enugh data the Machine Learning can not create a good prediction

get_ipython().run_cell_magic('time', '', 'LR_model_all = LR_classification(model_description = "All features LR",\n                    df = ML_BT,\n                    cat_columns = text_cat_cols + non_text_cat_cols, \n                    num_columns = text_num_cols + non_text_num_cols)\n\nLR_model_text = LR_classification(model_description = "Text features LR",\n                  df = ML_BT,\n                  cat_columns = text_cat_cols, \n                  num_columns = text_num_cols)\n\nLR_model_non_text = LR_classification(model_description = "Non-text features LR",\n                  df = ML_BT,\n                  cat_columns = non_text_cat_cols,\n                  num_columns = non_text_num_cols)')


###################ALL FEATURES LR##################
TRAIN AUC:       0.0
TEST AUC:        0.0
###################TEXT FEATURES LR##################
TRAIN AUC:       0.0
TEST AUC:        0.0
###################NON-TEXT FEATURES LR##################
TRAIN AUC:       0.0
TEST AUC:        0.0
Wall time: 7.47 s


In [23]:
# Because the parquet in data preperation is not created correctly with enugh data the Machine Learning can not create a good prediction

results

,algorithm,train AUC,test AUC
0,All features LR,0.0,0.0
1,Text features LR,0.0,0.0
2,Non-text features LR,0.0,0.0


In [24]:
results.to_csv("D:/School/STAGE_BP/ipo/results.csv")

In [25]:
def RF_classification(model_description, df, cat_columns, num_columns, output_column = "retweet_binary"):
    stringIndexers = []; indexed_cols = []; encoders = []; encoded_cols = []

    for column in cat_columns:
        inCol= column
        outCol = column + "indexed"
        stringIndexer = StringIndexer(inputCol = inCol, outputCol=outCol)
        stringIndexers.append(stringIndexer)
        indexed_cols.append(outCol)

    for column in indexed_cols:
        inCol= column
        outCol = column + "encoded"
        encoder = OneHotEncoder(inputCol=inCol, outputCol=outCol)
        encoders.append(encoder)
        encoded_cols.append(outCol)

    binarizer = Binarizer(inputCol = "retweet_count_double", outputCol = output_column, threshold=0.9)
    assembler = VectorAssembler(inputCols= num_columns + encoded_cols, outputCol="features")
    
    pipeline = Pipeline(stages= stringIndexers + encoders + [assembler] + [binarizer])
    pipelineModel = pipeline.fit(df)
    
    transformed_df = (pipelineModel.transform(df)
                                   .select("features", col("retweet_binary").cast("integer"), "retweet_count")
                     )
    
    train, validation, test = transformed_df.randomSplit([0.8, 0.1, 0.1], seed = 42)
    
    rf = RandomForestClassifier(numTrees=500, labelCol="retweet_binary")
    rf_model = rf.fit(train)

    train_predictions = rf_model.transform(train)
    validation_predictions = rf_model.transform(validation)
    
    AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="retweet_binary", metricName="areaUnderROC")

    AUC_train = AUC_evaluator.evaluate(train_predictions)
    AUC_validation = AUC_evaluator.evaluate(validation_predictions)
    print("###################" + model_description.upper() + "##################")
    print("TRAIN AUC:       {}".format(AUC_train))
    print("VALIDATION AUC:        {}".format(AUC_validation))